In [8]:

# from pypdf import PdfReader

# file_path = r"C:\Projects\Python\Stocks\Data\March_2024.pdf"

# # Creating a pdf reader object
# reader = PdfReader(file_path)

# # Extracting text from all pages
# all_text = ""
# for page in reader.pages:
#     all_text += page.extract_text()

# # Split the text into lines
# lines = all_text.split("\n")

# # Try to extract table-like data (you may need to adjust the splitting logic based on how the table is formatted)
# for line in lines:
#     # Example: Split by spaces or tabs to create rows and columns
#     # Adjust this based on the actual structure of your table
#     columns = line.split()
#     print(columns)


In [ ]:
import os
from pypdf import PdfReader

folder_path = r"C:\Projects\Python\Stocks\Data"  # Replace with your folder path

# # Check if the folder exists
# if os.path.exists(folder_path) and os.path.isdir(folder_path):
#     # List all files and directories inside the folder
#     files_and_folders = os.listdir(folder_path)
    
#     # Print the contents
#     for item in files_and_folders:
#         # print(item)
# else:
#     print("The folder does not exist.")


file_path = r"C:\Projects\Python\Stocks\Data\June_2024.pdf"  # Full file path

reader = PdfReader(file_path)
print(f"Number of pages: {len(reader.pages)}")
    
# Accessing the first page
page = reader.pages[1]
print(page.extract_text())

In [2]:
import numpy as np 
import re

flag = False 
i = 0

read_values = np.array([])
for lines in page.extract_text().split("\n"):
    # print(f"this is one line {lines} + index value: {i}")
    # i = i + 1
    # read_values = np.append(read_values,lines)
    if lines.__contains__("Activity - Current period"):
        flag = True
        # print("this is ti ")

    if flag == True: 
        read_values = np.append(read_values,lines)


In [3]:

parsed = read_values[1:2]

for value in parsed: 
        col_headings = re.split(r'(?=[A-Z])', value)

col_headings = [x for x in col_headings if x]
        
print(col_headings)


['Date ', 'Transaction', 'Description ', 'Charged ($)', 'Credit ($)', 'Balance ($)']


In [4]:

parse = np.array(read_values[2:])

print(type(read_values))

print(f"This is the lenght of the data points: {len(parse)}")

print(f"This is parse type: {type(parse)}")


<class 'numpy.ndarray'>
This is the lenght of the data points: 26
This is parse type: <class 'numpy.ndarray'>


In [5]:
parse = list(parse)

data = {
    "div": [],
    "buy": [],
    "cont": [],
    "lend": []
}

index = 0 

while index < len(parse) - 1:
    if "DIV" in parse[index]:
        if index + 2 < len(parse):  
            combined_text = parse[index] + " " + parse[index+1] + " " + parse[index+2]
            # del parse[index+1]
            # del parse[index+1]  # Since list shifted, index+1 now refers to old index+2

        data["div"].append(combined_text)  
 
    elif "BUY" in parse[index]:
        if "$" in parse[index]: 
            data["buy"].append(parse[index])
        else: 
            if index + 2 < len(parse):
                combined_text = parse[index] + " " + parse[index + 1] + " "+ parse[index + 2]
                # del parse[index+1]
                # del parse[index+1]
                
            data["buy"].append(combined_text)   

    elif "CONT" in parse[index]:
        if "$" in parse[index]: 
            data["cont"].append(parse[index])
        else: 
            if index + 2 < len(parse):
                combined_text = parse[index] + " " + parse[index + 1] + " "+ parse[index + 2]
                # del parse[index+1]
                # del parse[index+1]
                
            data["cont"].append(combined_text) 
    
    elif "FPLINT" in parse[index]:
        if index + 2 < len(parse):
            combined_text = parse[index] 
            # del parse[index+1]

        data["lend"].append(combined_text)
    
    index += 1  

# print("Div Data:", data["div"])
print("DIV Data:")
for item in data["div"]:
    print(item)

print("BUY Data:")
for item in data["buy"]:
    print(item)

print("CONT Data:")
for item in data["cont"]:
    print(item)

print("LEND Data:")
for item in data["lend"]:
    print(item)

# Convert lists to NumPy arrays if needed
# data["div"] = np.array(data["div"], dtype=object)


DIV Data:
2024-06-07DIV VDY - Vanguard FTSE Canadian High Dividend Yield Index ETF: Stock dividend distribution, received at 2024-06-07 $0.00 $0.18 $553.27
2024-06-14DIV PZA - Pizza Pizza Royalty Corp: Stock dividend distribution, received at 2024-06-14 $0.00 $0.23 $49.11
2024-06-14DIV PZA - Pizza Pizza Royalty Corp: Stock dividend distribution, received at 2024-06-14 $0.00 $0.08 $49.11
2024-06-28DIV CDZ - iShares S&P/TSX Canadian Dividend ETF: Stock dividend distribution, received at 2024-06-28 $0.00 $0.11 $49.22
2024-06-28DIV XEQT - iShares Core Equity ETF Portfolio: Stock dividend distribution, received at 2024-06-28 $0.00 $0.44 $49.66
BUY Data:
2024-06-10BUY ZEB - BMO Equal Weight Banks Index ETF: Bought 2.0000 shares (executed at 2024-06-07) $71.84 $0.00 $481.43
2024-06-10BUY L - Loblaw Cos. Ltd.: Bought 2.0000 shares (executed at 2024-06-07)$323.72 $0.00 $157.71
2024-06-10BUY SCHD - Schwab US Dividend Equity ETF: Bought 1.0000 shares (executed at 2024-06-07), FX Rate: 1.3984 $108

Database Structure: 
https://chatgpt.com/share/67ce0e2f-716c-8006-9043-925e27e44ce5

In [6]:
import sqlite3

database = sqlite3.connect("stock_database.db")  # creating db if it doesnt exist 
cursor = database.cursor() # object to interact with the database 

"""
Create db with following column headers: 

PDF Columns: date, type, Name, Quantity, value, Credit, Balance

Query Dividends: Date, Ticker, Company, Desc, recv_date, Value, Credit, Balance
Query Buy: Date, Ticker, Desc, Bought, edate, Fx_rate, Value, Credit, Balance
Query Contributions: Date, value, Credit, Balance
Query Lending: Date, Value, Credit, Balance

  
"""
# Creating Database Schema
cursor.execute('''
CREATE TABLE IF NOT EXISTS Stocks (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    Ticker TEXT UNIQUE NOT NULL,
    Company TEXT NOT NULL
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Transactions(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    Stock_ID INTEGER NOT NULL,
    Date TEXT NOT NULL, 
    Type TEXT CHECK (Type IN ('Buy', 'Sell', 'Dividend', 'Contribution', 'Lending')),
    Executed_Date TEXT NOT NULL,
    Shares REAL,
    Fx_Rate REAL, 
    Value REAL NOT NULL,
    Credit REAL NOT NULL,
    Balance REAL NOT NULL,
    FOREIGN KEY (Stock_ID) REFERENCES Stocks(id) ON DELETE CASCADE
);
''')

# Commit changes and close database
database.commit()
database.close()



In [7]:
import re 

print("Dividend data")
for item in data["div"]:

    # Regular Expression with Capture Groups
    pattern = r'(\d{4}-\d{2}-\d{2})DIV\s(\w+)\s-\s(.+):\s(.+),\sreceived at (\d{4}-\d{2}-\d{2})\s\$(\d+\.\d+)\s\$(\d+\.\d+)\s\$(\d+\.\d+)'

    match = re.match(pattern, item)

    if match:
        date, ticker, company, desc, recv_date, amount1, amount2, amount3 = match.groups()
        print(f"Date: {date}")
        print(f"Ticker: {ticker}")
        print(f"Company: {company}")
        print(f"Description: {desc}")
        print(f"Received Date: {recv_date}")
        print(f"Amounts: ${amount1}, ${amount2}, ${amount3}")
        print("\n")
    else:
        print("No match found for:", item)  # Debugging

print("Purchases Data")
for item in data["buy"]:

    # Adjusted regex pattern
    pattern = r'(\d{4}-\d{2}-\d{2})BUY\s(\S+)\s-\s(.+?):\sBought\s(\d+\.\d+)\sshares\s\(executed\sat\s(\d{4}-\d{2}-\d{2})\)(?:,\sFX Rate:\s([\d.]+))?\s?\$(\d+(?:\.\d+)?)\s\$(\d+(?:\.\d+)?)\s\$(\d+(?:\.\d+)?)'

    match = re.match(pattern, item)

    if match:
        groups = match.groups()
        date, ticker, desc, bought, edate, fx_rate, amount1, amount2, amount3 = groups
        print(f"Date: {date}")
        print(f"Ticker: {ticker}")
        print(f"Description: {desc}")
        print(f"Bought: {bought} shares")
        print(f"Executed Date: {edate}")
        if fx_rate:
            print(f"FX Rate: {fx_rate}")
        print(f"Amounts: ${amount1}, ${amount2}, ${amount3}")
        print("\n")
    else:
        print("No match found for:", item)  # Debugging

# print("Contributions Data")
# for item in data["cont"]:

#     pattern = r'(\d{4}-\d{2}-\d{2})(?:CONT Contribution\s\(executed at \d{4}-\d{2}-\d{2}\))\s\$(\d+\.\d+)\s\$(\d+\.\d+)\s\$(\d+\.\d+)'


#     match = re.match(pattern, item)

#     if match:
#         groups = match.groups()
#         date, amount1, amount2, amount3 = groups
#         print(f"Date: {date}")
#         print(f"Amounts: ${amount1}, ${amount2}, ${amount3}")
#     else:
#         print("No match found for:", item)  # Debugging


print("Contributions Data")
for item in data["cont"]:

    pattern = r'(\d{4}-\d{2}-\d{2})(?:CONT Contribution\s\(executed at \d{4}-\d{2}-\d{2}\))\s\$(\d+\.\d+)\s\$(\d+\.\d+)\s\$(\d+\.\d+)'

    match = re.match(pattern, item)

    if match:
        groups = match.groups()
        date, amount1, amount2, amount3 = groups
        print(f"Date: {date}")
        print(f"Amounts: ${amount1}, ${amount2}, ${amount3}")
    else:
        print("No match found for:", item)  # Debugging
    print("\n")


print("Stock Lending")
for item in data["lend"]: 
    
    pattern = r'(\d{4}-\d{2}-\d{2})FPLINT\s.*?\$(\d+\.\d+)\s+\$(\d+\.\d+)\s+\$(\d+\.\d+)'

    match = re.match(pattern, item)

    if match: 
        groups = match.groups()
        date, amount1, amount2, amount3 = groups
        print(f"Date: {date}")
        print(f"Amounts: ${amount1}, ${amount2}, ${amount3}")
    else: 
        print("No match found for:", item)  # Debugging

Dividend data
Date: 2024-06-07
Ticker: VDY
Company: Vanguard FTSE Canadian High Dividend Yield Index ETF
Description: Stock dividend distribution
Received Date: 2024-06-07
Amounts: $0.00, $0.18, $553.27


Date: 2024-06-14
Ticker: PZA
Company: Pizza Pizza Royalty Corp
Description: Stock dividend distribution
Received Date: 2024-06-14
Amounts: $0.00, $0.23, $49.11


Date: 2024-06-14
Ticker: PZA
Company: Pizza Pizza Royalty Corp
Description: Stock dividend distribution
Received Date: 2024-06-14
Amounts: $0.00, $0.08, $49.11


Date: 2024-06-28
Ticker: CDZ
Company: iShares S&P/TSX Canadian Dividend ETF
Description: Stock dividend distribution
Received Date: 2024-06-28
Amounts: $0.00, $0.11, $49.22


Date: 2024-06-28
Ticker: XEQT
Company: iShares Core Equity ETF Portfolio
Description: Stock dividend distribution
Received Date: 2024-06-28
Amounts: $0.00, $0.44, $49.66


Purchases Data
Date: 2024-06-10
Ticker: ZEB
Description: BMO Equal Weight Banks Index ETF
Bought: 2.0000 shares
Executed Dat

list of things  that will need to completed: 
- extract information from the data dictionary getting all the important information from there 
- setting up the firebase data base ( or set up a local server like SQLite )
-   This is the link to the prompt to set up https://chatgpt.com/share/67c6430a-ecd4-8006-8888-111af83a6fe2

Thoughts as we I go thru 
- what information do I plan on having saved in the the DB and what are the dates for this movign forword 
- for purchases I will need to see the amount that was purchased (quantity)